In [4]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import pandas as pd
import random
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf


In [5]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
        except RuntimeError as e:
            print(e)
print("Completed search for GPUs")

Completed search for GPUs


In [ ]:
def preprocess_data(x_train,y_train,x_test,y_test,subtract_pixel=True):
    x_train = x_train.astype('float32')/255
    x_test = x_test.astype('float32')/255
    
    #Getting the mean of each pixel across the entire dataset 
    if subtract_pixel:
        x_train_mean = np.mean(x_train,axis=0)
        x_train -= x_train_mean
        x_test -= x_test_mean
    return x_train, y_train, x_test, y_test

def rotate_image(im, iterations=None):
    if not iterations:
        iterations = random.randint(0,3)
    #Keep track of what angle the image has been rotated to
    y = [0,0,0,0]
    y[iterations] = 1
    for i in range(iterations):
        im = np.rot90(im)
    return im, y


def lr_schedule(epoch):
    lr = 1e-3
    if epoch >180:
        le *=0.5e-3
    elif epoch >160:
        le *=1e-3
    elif epoch >120:
        le *=1e-2
    elif epoch >80:
        le *=1e-1
    print("Learning Rate: ", lr)
    return lr

In [ ]:
def resnet_layer(inputs, num_filters=16, 
                 kernel_size=3, 
                 strides=1, 
                 activation='relu', 
                 batch_normalization=True, 
                 conv_first=True):

    #2D conv batch normalized activation stack builder
    #output is tensor as input for next layer
    conv = Conv2D(num_filters, 
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initialzier='he_normal'
                  kernel_regularizer=l2(le-4))
    x = inputs
    
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)    
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_classes=10):
    if (depth-2)%6 !=0:
        raise ValueError("Depth should be 6n+2")
    num_filters = 16
    num_res_blocks = int((depth-2)/6)
    
    #instantiate a keras object
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs = inputs)
    
    #instantate stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks): 
            strides=1
            if stack > 0 and res_block == 0: #first layer but not first stack
                strides = 2  #downsample
                
            #For each layer in each block
            y = resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
            y = resnet_layer(inputs=y, num_filters=num_filters, strides=strides)
            
            if stack > 0 and res_block == 0:    #first layer but not first stack
                #linear projection residual shortcut connection to match
                #changed dims
                x = resnet_layer(inputs=y,
                                num_filters=num_filters,
                                kernel_size=1,
                                strides=strides,
                                activation=None
                                batch_normalization=False)
                x = keras.layers.add([x,y])
                x = Activation('relu')(x)
            num_filters *= 2
        
        #Add classifier on top
        x = AveragePooling2D(pool_size=8)(x)
        y = Flatten()(x)
        outputs = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(y)
        output_model = Model(inputs=inputs,outputs=outputs)
        return output_model

In [ ]:
def make_rotated_data(Train, Test, subtract_pixel_mean=True):
    model_name='cifar100SSL'
    xy_rot_train = list(zip(*[rotate_image(im) for im in Train[0]]))
    xy_rot_test = list(zip(*[rotate_image(im) for im in Test[0]]))
    
    #x_rot refers to rotated image
    #x_rot refers to how much it was rotated (0,90,180,270)
    
    x_rot_train = np.array(xy_rot_train[0][:]).astype('float32'/255)
    t_rot_train = np.array(xy_rot_train[1][:])
    
    x_rot_ttest = np.array(xy_rot_test[0][:]).astype('float32'/255)
    y_rot_train = np.array(xy_rot_test[1][:])
    
    if subtract_pixel_mean:
        x_rot_train_mean = np.mean(x_rot_train, axis=0)
        x_rot_train -= x_rot_train_mean 
        x_rot_test -= x_rot_test_mean
    
    return x_rot_train, y_rot_train, x_rot_test, y_rot_test